In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten

from loader import load_20news
from custom_layer import MonteCarloLRF

import numpy as np
import sklearn as sk

Using TensorFlow backend.


In [3]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=1000, sparse=False, remove_short_documents=True, verbose=False)

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
batch_size = 16
num_classes = Y_train.shape[1]

In [5]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(9922, 1000, 1)
(6695, 1000, 1)
16 20


In [6]:
METRIC = 'euclidean'
distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

# enforce exact zero
for k in xrange(distances.shape[0]):
  distances[k,k] = 0.

In [7]:
# max normalize
distances /= distances.max()

# use tricube kernel (becaause of flatness around 0)
probabilities = (1. - np.abs(distances) ** 3) ** 3

# remove auto connections (which are taken anyway in LRF)
for k in xrange(probabilities.shape[0]):
  probabilities[k,k] = 0.

# normalize proba
probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))

In [ ]:
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=256, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Conv1D(64, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.add(Dropout(0.2))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_1 (MonteCarl (None, 1000, 256)         4352      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 64)          16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1280020   
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
Total params: 1,300,820
Trainable params: 1,300,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 9922 samples, validate on 6695 samples
Epoch 1/20
9922/9922 [==============================] - 72s - loss: 4.5845 - acc: 0.3808 - val_loss: 1.6246 - val_acc: 0.5401
Epoch 2/20
9922/9922 [==============================] - 71s - loss: 3.9335 - acc: 0.5852 - val_loss: 1.6175 - val_acc: 0.5492
Epoch 3/20
9922/9922 [==============================] - 71s - loss: 3.8018 - acc: 0.6312 - val_loss: 1.7270 - val_acc: 0.5350
Epoch 4/20
9922/9922 [==============================] - 71s - loss: 3.6014 - acc: 0.6648 - val_loss: 1.8597 - val_acc: 0.5346
Epoch 5/20
9922/9922 [==============================] - 71s - loss: 3.6432 - acc: 0.6785 - val_loss: 1.9920 - val_acc: 0.5340
Epoch 6/20
9922/9922 [==============================] - 71s - loss: 3.5032 - acc: 0.6972 - val_loss: 2.0746 - val_acc: 0.5270
Epoch 7/20
9922/9922 [==============================] - 71s - loss: 3.5552 - acc: 0.7091 - val_loss: 2.1806 - val_acc: 0.5252
Epoch 8/20
9922/9922 [==============================] - 71s - loss: 3.